## Importing the necessary libraries

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_regression
from sklearn.metrics import r2_score

## Perpare the Model (Lasso Regression)

In [38]:
class Regression:
    def __init__(self, learning_rate, iterations, regularization_parameter):
        self.m = None
        self.n = None
        self.w = None
        self.b = None
        self.regularization_parameter = regularization_parameter
        self.learning_rate = learning_rate
        self.iterations = iterations

    def cost_function(self, y, y_pred):
        return (1/(2*self.m))*np.sum(np.square(y-y_pred)) + self.regularization_parameter(self.w)
        
    def hypothesis(self, weights, bias, X):
        return X.dot(weights) + bias

    def train(self, X, y):
        X = np.insert(X, 0, 1, axis=1)
        try:
            y.shape[1]
        except IndexError as e:
            print("ERROR: Target array should be a one dimensional array not a list"
                  "----> here the target value not in the shape of (n,1). \nShape ({shape_y_0},1) and {shape_y} not match"
                  .format(shape_y_0=y.shape[0], shape_y=y.shape))
            return
        self.m = X.shape[0]
        self.n = X.shape[1]
        self.w = np.zeros((self.n, 1))
        self.b = 0

        for it in range(1, self.iterations + 1):
            y_pred = self.hypothesis(self.w, self.b, X)
            cost = self.cost_function(y, y_pred)
            dw = (1/self.m) * np.dot(X.T, (y_pred - y)) + self.regularization_parameter.derivation(self.w)
            self.w = self.w - self.learning_rate * dw

            if it % 10 == 0:
                print("The Cost function for the iteration {}----->{} :)".format(it, cost))

    def predict(self, test_X):
        test_X = np.insert(test_X, 0, 1, axis=1)
        y_pred = self.hypothesis(self.w, self.b, test_X)
        return y_pred

In [39]:
class l1_regularization:
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        return self.lamda * np.sum(np.abs(weights))
    
    def derivation(self, weights):
        return self.lamda * np.sign(weights)
    

class l2_regularization:
    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        return self.lamda * np.sum(np.square(weights))
    
    def derivation(self, weights):
        return self.lamda * 2 * (weights)

## Create the dataset

In [40]:
X,y=make_regression(n_samples=80000, n_features=8)

y=y[:,np.newaxis]

print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))
print("Shape of the target value ----------> {}".format(y.shape))

Number of training data samples-----> 80000
Number of training features --------> 8
Shape of the target value ----------> (80000, 1)


In [41]:
df=pd.DataFrame(X)
df.head()

,0,1,2,3,4,5,6,7
0,-1.115864,0.874499,1.586169,-1.483878,0.200694,0.383530,-0.917942,-0.751221
1,-1.827163,-1.957370,0.583514,0.361739,1.239281,1.085531,0.004975,-0.405198
2,1.933050,1.229018,-1.716412,-0.134172,-1.129719,-0.523720,0.152250,0.042619
3,3.192906,0.275112,-0.231609,-0.152483,-0.122628,0.787526,0.445755,-2.056910
4,1.053677,1.799800,-1.151786,2.315823,-1.480508,0.619357,-0.039033,0.245762


In [42]:
df_y=pd.DataFrame(y)
df_y.head()

,0
0,-75.468766
1,-83.965764
2,41.592487
3,90.168922
4,227.958708


In [43]:
class Lasso_Regression(Regression):
    def __init__(self,lamda,learning_rate,iterations):
        self.regularization = l1_regularization(lamda)
        super(Lasso_Regression, self).__init__(learning_rate, iterations, self.regularization)

    def train(self, X, y):
        return super(Lasso_Regression, self).train(X, y)
    
    def predict(self, test_X):
        return super(Lasso_Regression, self).predict(test_X)

## define the parameters and Model fit

In [44]:
param={
    "lamda":0.01,
    "learning_rate":0.01,
    "iterations":1000
}

linear_reg=Lasso_Regression(**param)

linear_reg.train(X,y)

y_pred=linear_reg.predict(X)

score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

The Cost function for the iteration 10----->12832.992924657494 :)
The Cost function for the iteration 20----->10505.044357439901 :)
The Cost function for the iteration 30----->8599.561615591605 :)
The Cost function for the iteration 40----->7039.8741102536505 :)
The Cost function for the iteration 50----->5763.22631407234 :)
The Cost function for the iteration 60----->4718.25218381324 :)
The Cost function for the iteration 70----->3862.9079945251237 :)
The Cost function for the iteration 80----->3162.7803763412185 :)
The Cost function for the iteration 90----->2589.7014494118125 :)
The Cost function for the iteration 100----->2120.615314934108 :)
The Cost function for the iteration 110----->1736.6502785540129 :)
The Cost function for the iteration 120----->1422.3594639625835 :)
The Cost function for the iteration 130----->1165.0992527362318 :)
The Cost function for the iteration 140----->954.5205342800864 :)
The Cost function for the iteration 150----->782.1522904890837 :)
The Cost fun

## lets verify from pre defined library

In [45]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

print("Number of training data samples-----> {}".format(X.shape[0]))
print("Number of training features --------> {}".format(X.shape[1]))

Number of training data samples-----> 80000
Number of training features --------> 8


In [46]:
lasso_sklearn = Lasso()
lasso_sklearn.fit(X, y)

y_pred_sklearn = lasso_sklearn.predict(X)

score_sklearn = r2_score(y, y_pred_sklearn)
print("The r2_score of the trained model", score_sklearn)

The r2_score of the trained model 0.999738756695463


## Perpare the Model (Ridge Regression)

In [47]:
class Ridge_Regression(Regression):
    def __init__(self,lamda,learning_rate,iterations):
        self.regularization = l2_regularization(lamda)
        super(Ridge_Regression, self).__init__(learning_rate, iterations, self.regularization)

        def train(self, X, y):
            return super(Ridge_Regression, self).train(X, y)
        
        def predict(self, test_X):
            return super(Ridge_Regression, self).predict(test_X)

## define the parameters and Model fit

In [48]:
param={
    "lamda":0.01,
    "learning_rate":0.01,
    "iterations":1000
}

linear_reg=Ridge_Regression(**param)

linear_reg.train(X,y)

y_pred=linear_reg.predict(X)

score = r2_score(y, y_pred)
print("The r2_score of the trained model", score)

The Cost function for the iteration 10----->12836.490214718964 :)
The Cost function for the iteration 20----->10520.590270697554 :)
The Cost function for the iteration 30----->8632.613767831617 :)
The Cost function for the iteration 40----->7093.487279430373 :)
The Cost function for the iteration 50----->5838.7495023387955 :)
The Cost function for the iteration 60----->4815.850941493497 :)
The Cost function for the iteration 70----->3981.9526330924414 :)
The Cost function for the iteration 80----->3302.131676341332 :)
The Cost function for the iteration 90----->2747.918390006724 :)
The Cost function for the iteration 100----->2296.1038055884105 :)
The Cost function for the iteration 110----->1927.7675361916154 :)
The Cost function for the iteration 120----->1627.485293854467 :)
The Cost function for the iteration 130----->1382.6828551386498 :)
The Cost function for the iteration 140----->1183.109410662881 :)
The Cost function for the iteration 150----->1020.4082360847535 :)
The Cost fu

## lets verify from pre defined library

In [49]:
from sklearn.linear_model import Ridge

ridge_sklearn = Ridge()
ridge_sklearn.fit(X, y)

y_pred_sklearn = ridge_sklearn.predict(X)

score_sklearn = r2_score(y, y_pred_sklearn)
print("The r2_score of the trained model", score_sklearn)

The r2_score of the trained model 0.9999999998425447
